In [1]:
import pandas as pd
import os
import numpy as np

# Dataset

In [22]:
data_file = r"1105e/FullDataset/Final_data_1105e.pkl"
data = pd.read_pickle(data_file)

In [13]:
data

GazeX     GazeY      GazeZ  Recording timestamp   target picture  \
0      152.0096   20.2408   497.4997                   20  AG Null    Null   
1       91.5267  -15.7177   365.1143                   40  AG Null    Null   
2      189.4057   -9.6084   886.4117                   60  AG Null    Null   
3      170.1867  -26.8524   509.0201                   80  AG Null    Null   
4      169.8932  -27.7286   369.5690                  100  AG Null    Null   
...         ...       ...        ...                  ...      ...     ...   
58293  -35.7709  288.0234  1090.1591              1253432  AG Null    Null   
58294  -58.2455  247.9092  1099.1627              1253452  AG Null    Null   
58295  -19.8429    1.7944   273.0716              1253472  AG Null    Null   
58296  -69.3203  -23.4384   262.0244              1253492  AG Null    Null   
58297  -42.7504 -110.5810   343.8976              1253572  AG Null    Null   

           AccX       AccY      AccZ      GyroX       GyroY      GyroZ  
0      0.674664  -9.787647  2.612879  10.770769  106.059807   0.904581  
1      0.385073  -9.946392  1.999057  15.626822  133.458525   1.011099  
2     -0.979491  -9.978140  2.121815  12.625428  110.787960  -7.364873  
3     -0.732546  -9.611419  2.078551  16.008721  116.409430 -34.749907  
4      0.333401  -9.609714  1.900401  20.392657   98.363320 -45.158527  
...         ...        ...       ...        ...         ...        ...  
58293 -1.192481 -10.129268 -1.133554  28.868055  107.195244  -7.847105  
58294 -1.218537 -10.214132 -1.161397  35.098475  101.830436  -2.509971  
58295 -1.441757 -10.147727 -1.084285  37.155527  102.776885  10.210608  
58296 -1.544585 -10.011470 -1.112609  34.284361   98.902668  13.367851  
58297 -1.682808  -9.905557 -1.071525  35.652618   99.358541  10.078360  

[58298 rows x 12 columns]

In [23]:
def windows(data, size,factor=2):
    start = 0
    while start + (size / factor) < len(data):
        yield int(start), int(start + size)
        start += (size / factor)


In [24]:
def to_seconds(time,factor = 1000):
    seconds = time/factor
    return round(seconds,2)

In [25]:
def get_pictures_dataset(data,user,window_size,factor =1):
    df_pictures = {"id":[],"target":[],"start":[],"end":[]}
 
    for idx,(start, end) in enumerate(windows(data.index, window_size,factor)):
        df_pictures["id"].append(f"{user}_{idx}")
        df_pictures["target"].append(data.loc[start:end,"picture"].mode()[0])
        df_pictures["start"].append(to_seconds(data.loc[start,"Recording timestamp"]))
        df_pictures["end"].append(to_seconds(data.loc[end,"Recording timestamp"]))
        
    return pd.DataFrame.from_dict(df_pictures).set_index("id")

In [31]:
user = "1105e"
test = get_pictures_dataset(data,user,500)

In [37]:
samples_max=max(test[test["target"]!="Null"]
                .target \
                .value_counts() )                

In [76]:
def filter_nulls(df):
    #get samples of majoritary class different than null
    samples_max=max(test[test["target"]!="Null"]
                    .target \
                    .value_counts() ) 
    #get id for null and rest of labels
    null_idx=df[df["target"]=="Null"].index
    samples_null = len(null_idx)
    if(samples_null>samples_max):
        #perform random undersampling
        rest_idx = df[df["target"]!="Null"].index
        # sample a subset of id from null samples to match majoritary class samples
        selected_idx = np.random.choice(range(samples_null), size=samples_max, replace=False)
        null_idx=null_idx[selected_idx]
        #filter df 
        df = df.loc[null_idx|rest_idx]

    return df
    

In [77]:
df= filter_nulls(test)

In [78]:
df

target    start      end
id                                   
1105e_100  _Y3A6113  1063.60  1073.96
1105e_101  _Y3A6113  1073.96  1084.25
1105e_11   _Y3A6125   118.23   131.36
1105e_16   _Y3A6096   174.14   184.51
1105e_22   _Y3A6093   239.74   249.98
1105e_24   _Y3A6092   261.07   271.16
1105e_25   _Y3A6092   271.16   281.16
1105e_26   _Y3A6092   281.16   291.35
1105e_29   _Y3A6090   312.78   322.80
1105e_3    _Y3A6132    33.32    44.81
1105e_39       Null   416.00   426.88
1105e_4    _Y3A6132    44.81    55.57
1105e_40       Null   426.88   437.95
1105e_42   _Y3A6098   448.40   458.62
1105e_50       Null   532.13   542.17
1105e_51       Null   542.17   552.42
1105e_52   _Y3A6101   552.42   562.48
1105e_61       Null   650.26   660.84
1105e_64   _Y3A6103   681.53   691.64
1105e_65   _Y3A6103   691.64   702.48
1105e_66   _Y3A6103   702.48   712.51
1105e_68   _Y3A6105   724.84   734.98
1105e_69   _Y3A6105   734.98   744.99
1105e_70   _Y3A6105   744.99   755.80
1105e_72   _Y3A6110   765.92   776.79
1105e_73   _Y3A6110   776.79   786.81
1105e_74   _Y3A6110   786.81   796.88
1105e_75   _Y3A6110   796.88   807.03
1105e_76   _Y3A6110   807.03   818.03
1105e_8    _Y3A6132    86.93    97.10
1105e_9    _Y3A6132    97.10   107.34
1105e_95   _Y3A6113  1009.86  1020.35

# Video

In [79]:
from moviepy.editor import *

In [95]:
path_video = r"1105e/FullDataset/1105e.mp4"
video = VideoFileClip(path_video).resize((224,224))

In [89]:
df = df.reset_index()

In [96]:
start = df.loc[0,"start"]
end =df.loc[0,"end"]

clip = video.subclip(start,end)
n_frames  = clip.duration * clip.fps

In [97]:
# Write the result to a file (many options available !)
clip_id = df.loc[0,"id"]
clip_file= f"{clip_id}.mp4"
clip.write_videofile(clip_file,audio=False,logger = None)

In [19]:
video = VideoFileClip("myHolidays_edited_2.mp4")

In [21]:
video.reader.nframes/25

10.04